In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import random

In [2]:
iris=pd.read_csv('iris.csv')

In [3]:
iris.columns

Index(['Unnamed: 0', 'sepal_lenght', 'sepal_width', 'petal_lenght',
       'petal_width', 'class'],
      dtype='object')

In [4]:
iris.head()
iris=iris.drop(columns='Unnamed: 0')
iris.head()

,sepal_lenght,sepal_width,petal_lenght,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
iris=iris.rename(columns={'sepal_lenght':'sepal_len','petal_lenght':'petal_len'})

In [6]:
iris.head()

,sepal_len,sepal_width,petal_len,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
y_class=list(iris['class'].unique())

In [8]:
def gaussian_distribution(value,data,col):
    variance=data[col].var()
    mean=data[col].mean()
    e=2.71828
    
    
    denom=(2*3.14*variance)**0.5   #root(2*pi*sigma^2)
    expo=((value-mean)**2)/(2*variance) #-(X-u)^2/2*sigma^2
    numer=e**(-expo)
    
    
    return numer/denom

In [9]:
# def categorical(value,data,col):
#     return new_df=(data[(data['class']==target)&(data[col]==value)].shape[0])/(data['class']==target).shape[0]

In [10]:
def compute_probablity(target,X_test,data):
    class_counts=dict(data['class'].value_counts())
    prob_y=class_counts[y]/sum(class_counts.values())
    
    X=data.drop(columns='class').columns
    
    new_df=data[data['class']==target]
    p=1
    for x_value,col in zip(X_test,X):
        p=p*gaussian_distribution(x_value,new_df,col)
    
    return (prob_y*p)
    

In [11]:
test_index=random.randint(0,iris.shape[0])
# test_index=26
print(test_index)
X_test=iris.iloc[test_index,:4]
y_act=iris.loc[test_index,'class']

print(X_test,y_act)

33
sepal_len      5.5
sepal_width    4.2
petal_len      1.4
petal_width    0.2
Name: 33, dtype: object Iris-setosa


In [12]:
probablities=dict()



for y in y_class:
    probablities[y]=compute_probablity(y,X_test,iris)

print(probablities)
# print(X_test)
y_predict=max(probablities,key=probablities.get)
print(y_act,y_predict)

{'Iris-setosa': 0.13244013910245467, 'Iris-versicolor': 1.0024968452750059e-20, 'Iris-virginica': 5.9158198631301835e-27}
Iris-setosa Iris-setosa


In [13]:
# data=pd.read_csv('HR_Analytics.csv')

# Naive Bayes

Prepare a Documentation for this Class and also Add link of notes/material used

In [285]:
class NaiveBayes():
    
    
    def __init__(self,train_data,target_column=-1):
        self.data=train_data
        self.X=train_data.iloc[:,:-1]
        self.y=train_data.iloc[:,-1]
        
        self.class_label=self.data.columns[-1]
        
        
    
    def predict(self,X_test,categorical=[]):
        
        self.categorical=categorical
#         print(self.categorical)
        
        self.X_test=X_test
        y_class=self.y.unique()
        probablity=dict()     
        
        for y in y_class:
            probablity[y]=self.calculate_probablity(y)
        y_predict=max(probablity,key=probablity.get)
                
        return y_predict
        
    def calculate_probablity(self,target):
        class_counts=dict(self.data.iloc[:,-1].value_counts())
        prob_y=class_counts[target]/sum(class_counts.values())
        
         #Columns with categorical can be passed as a list by default all sent as Guassian
        
        p=1
        new_df=self.data[self.data[self.class_label]==target]
        for xvalue,col in zip(self.X_test,self.X.columns):
            
#             print(xvalue,col,col in self.categorical)
            
            if col in self.categorical:
                p*=self.categorical_probablity(xvalue,new_df,col)
            if col not in self.categorical:
                p*=self.gaussian_probablity(xvalue,new_df,col)
        return (p*prob_y)
    
    def gaussian_probablity(self,value,new_df,col):
        variance=new_df[col].var()
        mean=new_df[col].mean()
        e=2.71828
                
        denom=(2*3.14*variance)**0.5   #root(2*pi*sigma^2)
        expo=((value-mean)**2)/(2*variance) #-(X-u)^2/2*sigma^2
        numer=e**(-expo)
        return numer/denom
    
    def categorical_probablity(self,value,new_df,col):
        nc=self.data[(self.data[self.class_label]==value)&(self.data[col]==value)].shape[0]
        n=(self.data[self.class_label]==value).shape[0]
        return (nc+1)/(n+len(self.data.iloc[:,-1].unique()))
    
    def score(self,X_test,y_actual,categorical=[]):
        score=0
        for y,test_row in zip(y_actual,X_test.values):
            if y==self.predict(test_row,categorical):
                score+=1
                
        print("Accuracy = ",100*(score/len(X_test)))
        return 100*(score/len(X_test))
    

In [286]:
from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)

X=iris.iloc[:,:-1]
y=iris.iloc[:,-1]

scores=[]
for train_index,test_index in kf.split(X,y):
    X_train=X.iloc[train_index]
    y_train=y.iloc[train_index]
    
    X_test=X.iloc[test_index]
    y_test=y.iloc[test_index]
    
    nb=NaiveBayes(iris.iloc[train_index])
    scores.append(nb.score(X_test,y_test))

print(np.mean(scores))

Accuracy =  93.33333333333333
Accuracy =  96.66666666666667
Accuracy =  93.33333333333333
Accuracy =  93.33333333333333
Accuracy =  100.0
95.33333333333333


In [287]:
df=pd.read_csv(r"D:\AI Adventures\Machine Learning\HR_Analytics.csv.crdownload")

In [288]:
df.loc[df['YearsWithCurrManager'].isnull(),'YearsWithCurrManager']=df['YearsWithCurrManager'].mean()
df=df.dropna()


In [289]:
df.head()

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,...,3,80,0,0,2,3,0,0.0,0.0,0.0
1,RM302,18,18-25,No,Travel_Rarely,812,Sales,10,3,Medical,...,1,80,0,0,2,3,0,0.0,0.0,0.0
2,RM458,18,18-25,Yes,Travel_Frequently,1306,Sales,5,3,Marketing,...,4,80,0,0,3,3,0,0.0,0.0,0.0
3,RM728,18,18-25,No,Non-Travel,287,Research & Development,5,2,Life Sciences,...,4,80,0,0,2,3,0,0.0,0.0,0.0
4,RM829,18,18-25,Yes,Non-Travel,247,Research & Development,8,1,Medical,...,4,80,0,0,0,3,0,0.0,0.0,0.0


In [290]:
Categorical=['Education','EnvironmentSatisfaction','JobInvolvement','JobLevel','JobSatisfaction','RelationshipSatisfaction',
            'StockOptionLevel','WorkLifeBalance']

df=df.drop(columns=['EmpID','EmployeeCount','Over18','PerformanceRating','StandardHours','EmployeeNumber','AgeGroup'])

In [291]:
df['Education'] = df['Education'].astype('category')


for col in Categorical:
    df[col]=df[col].astype('category')

In [292]:
df['target']=df['Attrition']
new_df=df.drop(columns='Attrition')
new_df


Categorical_columns=new_df.iloc[:,:-1].select_dtypes(['object','category']).columns
print(Categorical_columns)

Index(['BusinessTravel', 'Department', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'JobInvolvement', 'JobLevel',
       'JobRole', 'JobSatisfaction', 'MaritalStatus', 'SalarySlab', 'OverTime',
       'RelationshipSatisfaction', 'StockOptionLevel', 'WorkLifeBalance'],
      dtype='object')


In [293]:
from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)

X=new_df.iloc[:,:-1]
y=new_df.iloc[:,-1]
scores=[]

print(Categorical_columns)
# Categorical_columns=[]

for train_index,test_index in kf.split(X,y):
    X_test=new_df.iloc[test_index,:-1]
    y_test=new_df.iloc[test_index,-1]
    
    nb=NaiveBayes(new_df)
    scores.append(nb.score(X_test,y_test,categorical=Categorical_columns))
    
    
# nb=NaiveBayes(df,)

Index(['BusinessTravel', 'Department', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'JobInvolvement', 'JobLevel',
       'JobRole', 'JobSatisfaction', 'MaritalStatus', 'SalarySlab', 'OverTime',
       'RelationshipSatisfaction', 'StockOptionLevel', 'WorkLifeBalance'],
      dtype='object')
Accuracy =  51.973684210526315
Accuracy =  74.3421052631579
Accuracy =  79.47019867549669
Accuracy =  74.83443708609272
Accuracy =  74.17218543046357


In [294]:
print(np.mean(scores))

70.95852213314744
